In [5]:
import warnings
from importlib import reload

warnings.filterwarnings('ignore')

In [13]:
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

client = None

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import prompts
from importlib import reload

reload(prompts)

from prompts import TARRIFICARTOR_ADR

In [16]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, Field
from sqlalchemy import create_engine, Column, String, Integer, Float, DateTime, Text, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import uuid
import tiktoken

from ipywidgets import widgets, Layout, Button, Textarea, HBox
from IPython.display import display
import threading

completion_pricing_per_1k_tokens_usd = {
    "gpt-4-1106-preview": {"input": 0.01, "output": 0.03},
    "gpt-4-1106-vision-preview": {"input": 0.01, "output": 0.03},
    "gpt-4": {"input": 0.03, "output": 0.06},
    "gpt-4-32k": {"input": 0.06, "output": 0.12},
    "gpt-3.5-turbo-1106": {"input": 0.0010, "output": 0.002},
    "gpt-3.5-turbo-instruct": {"input": 0.0010, "output": 0.002},
}

assistants_api_price_usd = {
    "Code interpreter": {"input": 0.03},
    "Retrieval": {"input": 0.2},
}


# Define your Pydantic model for data validation
class PromptVersion(BaseModel):
    pkid: Optional[int] = None 
    id: Optional[str] = Field(default_factory=lambda: str(uuid.uuid4()))
    created_at: datetime = Field(default_factory=datetime.now)
    updated_at: datetime = Field(default_factory=datetime.now)
    prompt: str
    response: str
    model: str
    input_tokens: int
    output_tokens: int
    tags: Optional[str] = None
    total_price: float
    is_like: Optional[bool] = None
    temperature: Optional[float] = None
    feedback: Optional[str] = None

Base = declarative_base()

# Define your SQLAlchemy model for the database schema
class PromptVersionDB(Base):
    __tablename__ = 'prompt_versions'
    pkid = Column(Integer, primary_key=True, autoincrement=True)
    id = Column(String, default=lambda: str(uuid.uuid4()))
    created_at = Column(DateTime, default=datetime.now)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    prompt = Column(Text)
    response = Column(Text)
    model = Column(String)
    input_tokens = Column(Integer)
    output_tokens = Column(Integer)
    temperature = Column(Float)
    total_price = Column(Float)
    feedback = Column(Text)
    tags = Column(Text)
    is_like = Column(Boolean)


def num_tokens_from_string(string: str, encoding_name: str = "gpt-3.5-turbo") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
    

class GPTDatabaseLogger:
    def __init__(self, db_url):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)
        if os.getenv('OPENAI_API_KEY') is None:
            load_dotenv()
        self.client = openai.OpenAI()

    def create_completion(self, messages, temperature, output = True, tags: str = None,
                          model: str = 'gpt-4', **kwargs): 
        tokens_pricing = completion_pricing_per_1k_tokens_usd[model]

        prompt = "\n\n".join([msg['role'] + ": " + msg['content'] for msg in messages])
        prompt_tokens = sum([ num_tokens_from_string(msg['content']) for msg in messages])
    
        response = self.client.chat.completions.create(model=model, 
                                                  messages=messages, 
                                                  temperature=temperature, 
                                                  stream=True, **kwargs
                                                 )
        collected_messages = []
        for chunk in response:
            if chunk.choices[0].delta.content:
                if output:
                    print(chunk.choices[0].delta.content, end='')
                collected_messages.append(chunk.choices[0].delta.content)

        content_str = ''.join(collected_messages)
        output_tokens = num_tokens_from_string(content_str)

        total_price = (tokens_pricing['input'] * prompt_tokens + tokens_pricing['output'] * output_tokens) / 1000

        prompt_version = PromptVersion(
            prompt=prompt, 
            response=content_str, 
            model=model,
            temperature=temperature,
            input_tokens=prompt_tokens, 
            output_tokens=output_tokens,
            total_price=total_price,
            tags=tags
        )
        
        session = self.Session()
        db_record = PromptVersionDB(**prompt_version.dict())
        session.add(db_record)
        session.commit()
        self.note_id = db_record.id  # Assuming the record has an ID field
        session.close()

        # Step 3: Return result from method
        print("\n\n--------------------\n\nNote saved without feedback. ID:", self.note_id + '\n\n')

        # Step 4: Run the window for feedback form
        self.collect_feedback()

        return content_str

    
    def collect_feedback(self):
        feedback_input = Textarea(
            value='',
            placeholder='Type your feedback here...',
            description='Feedback:',
            disabled=False,
            layout=Layout(width='70%', height='80px')
        )

        like_button = Button(description='👍 Like', button_style='success', tooltip='Like this content')
        dislike_button = Button(description='👎 Dislike', button_style='danger', tooltip='Dislike this content')
        feedback_button = Button(description='Submit Feedback', button_style='success', tooltip='Click to submit feedback')

        def on_like_disliked(b):
            session = self.Session()
            note_to_update = session.query(PromptVersionDB).filter_by(id=self.note_id).first()
            if note_to_update:
                if b.description == '👍 Like':
                    note_to_update.is_like = True
                elif b.description == '👎 Dislike':
                    note_to_update.is_like = False
                session.commit()
            session.close()

        def on_feedback_submitted(b):
            feedback = feedback_input.value
            session = self.Session()
            note_to_update = session.query(PromptVersionDB).filter_by(id=self.note_id).first()
            if note_to_update:
                note_to_update.feedback = feedback
                session.commit()
                print("Feedback updated successfully.")
            else:
                print("Note not found.")
            session.close()
            feedback_input.value = ''  # Clear input after submission

        feedback_button.on_click(on_feedback_submitted)
        like_button.on_click(on_like_disliked)
        dislike_button.on_click(on_like_disliked)

        display(HBox([like_button, dislike_button]), feedback_input, feedback_button)

In [17]:
db_logger = GPTDatabaseLogger('sqlite:///../famaga/prompt_versions.db')

## Products

In [23]:
import prompts
from importlib import reload

reload(prompts)

from prompts import TARRIFICARTOR_ADR, ROUTES, ROUTES_DESCRIPTION

In [20]:
response = db_logger.create_completion([
    { "role": "user", "content": f"""
You are Senior Architect, you are usiung PlanUML and C4 to describe ADRs of Golang microservices. Could you please 
read the example of flow-overview.puml ADR and common HowTo that would describe how to create flow-overview diagrams
for other microservices, this should contains of basic concepts of approach and should be understandable Middle
Solutions Architect. Please read diagram description [DIAGRAM DESCRIPTIONS]. Read also flow overview of Senior Architect
it would be helpful and describing some approaches [FLOW-OVERVIEW].

Write guide how to create ADRs!

[DIAGRAM DESCRIPTIONS]
Here's a step-by-step guide on how to read this diagram:

1. The process starts with the ATS.hr-crm-api service (ATS) launching a task every 20 minutes to update the candidate's salary. It fetches all candidates who are not registered in WBTeam with the status "Processed/Working".

2. For each candidate, ATS sets the CalculationType to "IT department", MotivationType to "Undefined", and flags the candidate as registered in WBTeam. It then sends a request to WBTeam.crm-api (CRM) to update the candidate's salary.

3. CRM searches for a user by WBUserID or by (Fio AND Phone). It then requests the CustomFields service to save the motivation type in custom fields. The CustomFields service saves the motivation type and commits all changes.

4. CRM then requests the Tarificator service to save the CalculationType. The Tarificator service checks if the request type is not equal to 0. If it is not, it adds or updates the CalculationType in the tariff_office.employee_fields. If it is equal to 0, it searches for the calculation type in the custom_fields table. If there is no record, it adds a record to the table, gets the ID of CalculationType from the request, and adds or updates the CalculationType in the tariff_office.employee_fields. It then commits all changes.

5. CRM searches for a RECRUITER by WBUserID or by (Fio AND Phone), saves the salary for an employee with the status `send_to_tarificator` in the employee_income.income table, and updates the salary in the Tarificator service. It sends attributes including IsBonus: false, IsNewEmployee: true, and ApplicationDate: time.Now().

6. The Tarificator service updates the salary in the tariff_office.incomes table, sets the `calculation` and `payment` flags to true in the tariff_office.access_users table, sets the status of accruals in the tariff_office.accruals table, deletes canceled accruals, and commits all changes. It then sends a message to Tarificator.CALC for recalculation.

7. CRM requests the OrgStruct service to embed the user in the org-structure. The OrgStruct service adds or updates the group-employee relationship and commits all changes.

8. Finally, ATS commits the transaction.

This diagram is a great way to visualize the flow of data and interactions between different services in a microservice architecture. It helps in understanding the system's behavior and can be used as a reference for designing and implementing similar systems.
[/DIAGRAM DESCRIPTIONS]

[FLOW-OVERVIEW]
```
{TARRIFICARTOR_ADR}
```
[/FLOW-OVERVIEW]
"""}
], tags='adr', model='gpt-4', temperature=0.7) 

Creating Architectural Decision Records (ADRs) using PlantUML and C4 for Golang microservices involves several steps. Here's a step-by-step guide on how to create diagrams for other microservices:

Step 1: Define the Scope
Identify the microservice for which you are creating the ADR. Determine what processes, entities, and data flows are involved.

Step 2: Prepare the Flow-Overview Diagram
Using the example provided, prepare a flow-overview diagram for your microservice. This diagram should illustrate the process and data flow within the microservice. Use the PlantUML language to create this diagram.

Step 3: Detail Each Step
For each step in the process, provide a detailed explanation. Just like the example provided, describe what occurs at each stage, what data is involved, and which entities are interacting. This will help anyone reading the ADR to understand the logic and flow.

Step 4: Describe the Interactions
Detail the interactions between different services in your architectur

Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [ ]:
client_history = """
Date: 2023-07-24
425178 (R412022869) Aventics (brand of Emerson) R412022869 margin: 30.86%, sell: 23.04$ qty. 4

Date: 2023-10-17
446982 (CHV10L) ILME CHV10L margin: 24.94%, sell: 8.66$ qty. 5
446982 (CHV10LG) ILME CHV10LG margin: 24.93%, sell: 15.32$ qty. 5
446982 (CNEM 10T) ILME CNEM 10T margin: 25.0%, sell: 5.96$ qty. 5
446982 (CNEF 1OT) ILME CNEF 1OT margin: 24.92%, sell: 6.02$ qty. 5
446982 (CHV06L16) ILME CHV06L16 margin: 25.05%, sell: 5.31$ qty. 5
446982 (CHV06LG) ILME CHV06LG margin: 24.96%, sell: 14.18$ qty. 5
446982 (CNEM 06T) ILME CNEM 06T margin: 25.06%, sell: 4.47$ qty. 5
446982 (CNEF 06T) ILME CNEF 06T margin: 25.0%, sell: 4.56$ qty. 5

Date: 2023-10-31
451268 (T1040015C3) Tedea-Huntleigh (brand of VPG Transducers) T1040015C3 margin: 32.31%, sell: 121.88$ qty. 1

Date: 2023-11-16
455000 (E3010-013-005) Fraser Anti-Static E3010-013-005 margin: 22.23%, sell: 336.98$ qty. 2

Date: 2023-11-23
458446 (R412022869) Aventics (brand of Emerson) R412022869 margin: 55.16%, sell: 35.53$ qty. 2
"""

chain_of_thought = """
Thought: Let's start by analyzing the client's previous purchase history.
Block: [Block 1: Initial Contact]
Decision Point: Has the customer previously bought the same product?
Decision Observation: The client has not previously purchased the part END-Armaturen ZE311067. They have bought other products, but not this specific one.

Block Observation: Since the client hasn't purchased this product before, we need to understand the price they are willing to pay for it.

Thought: Now, we need to understand the price the client is willing to pay for the product.
Block: [Block 2: Customer Stated Desired Price]
Decision Point: Is there an answer with a specific price?
Decision Observation: The client has not stated a specific price they are willing to pay for the product.

Block Observation: Since the client hasn't stated a specific price, we need to return to the start of the cycle and ask the client for their desired price.

Thought: We need to ask the client for their desired price for the product.
Block: [Block 3: Return to the start of the cycle]
Decision Point: Has the customer previously bought the same product?
Decision Observation: The client has not previously purchased the part END-Armaturen ZE311067.
"""

client request

In [1]:
from prompts import TARRIFICARTOR_ADR

ImportError: cannot import name 'TARRIFICARTOR_ADR' from 'prompts' (C:\Users\MGroup\components_agent_sales\notebooks\famaga\prompts\__init__.py)

In [ ]:
import prompts.discounts.block_schema
from importlib import reload
import utils

reload(prompts.discounts.block_schema)
reload(utils)


from prompts.discounts.block_schema import correct_block_schema, generate_different_scenarious_by_decision_points, \
    generate_thread_metadata_by_scenario, company_system_message, example_scenarios_V1, \
    get_fields_for_scneraios_from_block_schema, RESULT_BLOCK_SCHEMA, generated_scenarios_v1, \
    continue_generating_different_scenarious_by_decision_points, generated_scenarios_without_points_v2, \
    example_scenarios_V2

from utils import get_scenarios


print('Updated')

In [25]:
response = db_logger.create_completion([
    { "role": "user", "content": f"""
You are Senior Architect, you are usiung PlanUML and C4 to describe ADRs of Golang microservices. Please
write flow-overview adr by the diagram description [DIAGRAM DESCRIPTIONS] and routes [ROUTES]. Please use example adr 
[FLOW-OVERVIEW] to create your own.

[DIAGRAM DESCRIPTIONS]
{ROUTES_DESCRIPTION}
[/DIAGRAM DESCRIPTIONS]

[ROUTE]
{ROUTES}
[/ROUTE]

[FLOW-OVERVIEW]
```
{TARRIFICARTOR_ADR}
```
[/FLOW-OVERVIEW]
"""}
], tags='adr', model='gpt-4', temperature=0.2) 

```

@startuml
!theme toy
scale 2

participant "API Router" as api
participant "Metrics Router" as metrics
participant "Middleware" as mid
participant "Service" as svc
participant "Logger" as log

group API
    api -> api: Initialization of API router handler
    api -> api: Define routes for health checks
    api -> api: Define routes for document operations
    api -> api: Define routes for photo and preview operations
    api -> api: Define routes for other services
    api -> api: Define legacy routes
    api -> api: Define routes for version 2 operations
    api -> api: Define routes for mobile requests
    api -> mid: Authenticate routes using middleware
    api -> svc: Call service methods based on routes
    api -> log: Log HTTP requests and responses
end

group Metrics
    metrics -> metrics: Initialization of Metrics router handler
    metrics -> metrics: Define "/metrics" route for monitoring performance metrics
    metrics -> log: Log HTTP requests and responses
end

@endum

Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [ ]:
This is an example of an Architecture Decision Record (ADR) described using PlantUML and C4 model. It represents the flow of data and interactions between different microservices in a Golang application. Here's a step-by-step guide on how to understand and create a similar ADR:

1. **Identify the Participants**: In this ADR, the participants are different microservices like ATS.hr-crm-api, WBTeam.cmr-api, CustomFields.Service, Tarificator.API, OrgStruct.Service, and NATS. These are represented using the `participant` keyword.

2. **Define the Interactions**: The arrows (`->`) represent the interactions between different participants. For example, `ats -> ats: Every 20 minutes a task to update candidate's salary is launched` means that the ATS.hr-crm-api service performs a task to update the candidate's salary every 20 minutes.

3. **Use Groups and Loops**: The `group` and `loop` keywords are used to represent a set of actions that are performed together or repeatedly. For example, the loop `for all candidates` means that the actions inside this loop are performed for each candidate.

4. **Handle Conditions**: The `alt` keyword is used to handle conditions. For example, `alt if request type is not equal to 0` means that the actions inside this block are performed if the request type is not equal to 0.

5. **Use Notes**: The `note over` keyword is used to add explanatory notes over a participant. For example, `note over crm` is used to add a note over the WBTeam.cmr-api service.

6. **Commit Changes**: The `commit all changes` statements represent the points where all the changes made by a service are committed.

7. **End the Diagram**: The `@enduml` keyword is used to signify the end of the UML diagram.

To create a similar ADR, you need to identify the participants (microservices) in your application, define their interactions, group related actions, handle conditions, and add explanatory notes. Remember to commit the changes made by each service and end the diagram with the `@enduml` keyword.